In [ ]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Please list the names of the facilities that do.
select name
from Facilities 
where membercost=0;

In [ ]:
#Q2: How many facilities do not charge a fee to members?
select count(facid)
from Facilities
where membercost=0;

In [ ]:
#Q3: How can you produce a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost?
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question.
SELECT facid,name,membercost,monthlymaintenance 
FROM `Facilities` 
WHERE membercost/monthlymaintenance <.2


In [ ]:
#Q4: How can you retrieve the details of facilities with ID 1 and 5?
#Write the query without using the OR operator.
SELECT * 
FROM Facilities
WHERE facid
IN ( 1, 5 ) 
LIMIT 0 , 30

In [ ]:
#Q5: How can you produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100? Return the name and monthly maintenance of the facilities
#in question.
SELECT name,monthlymaintenance,case when monthlymaintenance>100 then 'expensive'
else 'cheap' end 
as Cost_Category
from Facilities


In [ ]:
#You'd like to get the first and last name of the last member(s)
#who signed up. Do not use the LIMIT clause for your solution.
select firstname,surname 
from Members 
where joindate=(select max(joindate) from Members)

In [ ]:
#Q7: How can you produce a list of all members who have used a tennis court?
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name. 

select distinct(concat(Members.firstname," ",Members.surname," ",new.fac)) as MemberVisit from(
select Facilities.facid,Bookings.memid as mem,Facilities.name as fac from Bookings 
left join Facilities on Bookings.facid=Facilities.facid
where Facilities.name like 'Tennis%') as new
left join Members on new.mem=Members.memid
order by MemberVisit asc

In [ ]:
#Q8: How can you produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30? Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries.

select Facilities.name,concat(Members.firstname," ",Members.surname) as MemberName,case when Bookings.memid=0 then Bookings.slots*Facilities.guestcost else Bookings.slots*Facilities.membercost end as Cost
from Bookings
left join Facilities on Bookings.facid=Facilities.facid
left join Members on Bookings.memid=Members.memid
where left(Bookings.starttime,10)='2012-09-14'
having Cost>30
order by cost desc

In [ ]:
#Q9: This time, produce the same result as in Q8, but using a subquery. 

select ref2.facname,concat(Members.firstname," ",Members.surname) as member_name, ref2.Cost
From Members
right join
(select Facilities.name as facname,case when ref.memid=0 then Facilities.guestcost*ref.slots else Facilities.membercost*ref.slots end as Cost,ref.memid
from Facilities right join 
(select facid,slots,starttime,memid
from Bookings where left(starttime,10)='2012-09-14') as ref
on Facilities.facid=ref.facid
having cost>30) as ref2
on Members.memid=ref2.memid
order by Cost desc

In [ ]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! 
select Facilities.name as FacilityName,sum(case when Bookings.memid=0 then Facilities.guestcost*Bookings.slots
else Facilities.membercost*Bookings.slots end) as Revenue from Bookings
left join Facilities on Bookings.facid = Facilities.facid
group by Facilities.facid
having Revenue<1000
order by Revenue desc
